In [1]:
import json
import os
from torch.utils.data.dataset import random_split
import csv
from pytorch_transformers import BertTokenizer


/home/NewBio/wangzhuoyue/.conda/envs/cyg/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = BertTokenizer.from_pretrained(
         "../../backup/Robera/", do_lower_case=True) 

In [3]:
def list2tokens(strl,tokens,s_index):
    i=0
    t_index=0
    while i<s_index:
        if tokens[t_index]!='[UNK]':
            i+=len(tokens[t_index].strip('#'))
        else:
            i+=1
        t_index+=1
    return t_index


In [12]:
s='公诉机关指控，2013年11月以来，被告人张某某在辽阳市太子河区其租住的平房内多次向栗XX、王XX贩卖毒品。2014年2月在辽纺附近向刘XX贩卖500元冰毒。2014年4月在辽阳市白塔区外环路XX大院门前，张某某向刘XX贩卖500元冰毒。2014年4月，张某某在其租住的平房处向赵XX贩卖麻古15粒。2013年11月，张某某在其租住的平房处向魏XX贩卖500元冰毒。案发后，被告人张某某被公安机关抓获归案。公诉机关认为，被告人张某某的行为已触犯了《中华人民共和国刑法》第三百四十七条之规定，应以贩卖毒品罪追究其刑事责任。'
l=tokenizer.tokenize(s)
st=42
en=st+len("栗XX")
e=list2tokens(s,l,en)
print(s[st:en],e)
l[38:40+1]

栗XX 40


['栗', 'xx', '、']

In [4]:
# 使用bertTokenizer切词
test_=[]
with open('cail_drug.json','rt',encoding='utf-8') as fin2,\
open('test_cail.sent','w',newline='',encoding='utf-8') as ftes,\
open('test_cail.pointer','w',newline='',encoding='utf-8') as ftep:
       
    
    for line in fin2:
        
        line = line.strip()
        # print(line)
        if not line:
            continue  #结束则跳出循环
        try:
            sentence = json.loads(line)#将json数据转换成python对象  这里是转换为了字典
            test_.append(sentence)
        except:
            pass
    # print(test_)
    test_=test_[-100:]
    for te in test_:
        sentence_text= "".join(te["fact"].strip().strip('"').split())
        # print(sentence_text)
        if(len(sentence_text)>510):
            sentence_text =sentence_text[:510]
        ftes.write(sentence_text+'\n')
        sentence_l=tokenizer.tokenize(sentence_text)
        
        pointer=[]
        pointer.append(' '.join(['0','0','0','0','posess']))
        ftep.write(' | '.join(pointer)+'\n')


In [5]:
with open('test_cail.tup','w',newline='',encoding='utf-8') as ftet:
            
    for te in test_:
        tuples=[]
        tuples.append(' ; '.join(['e1','e2','posses']))
        ftet.write(' | '.join(tuples)+'\n')


In [6]:
# 使用list()切词，以字符为单位 BIOES
with open('test_cail.sent','rt',encoding='utf-8') as fin1,open('test_cail.pointer','rt',encoding='utf-8') as fin2,\
open('test_cail.ner','w',newline='',encoding='utf-8') as ftrs:
    count=0
#     train_=[]
    for line1,line2 in zip(fin1,fin2):
        line1 = line1.strip()
#         train_.append(line1)
        if len(line1)>256:
            count+=1
        line2=line2.strip().split('|')
        if not line1:
            continue  #结束则跳出循环
        res=['O'for i in range(len(line1))]
        
        ftrs.write(' '.join(res)+'\n')
           
    

In [7]:
import re
# 使用list()切词，以字符为单位  BIO
with open('test_cail.sent','rt',encoding='utf-8') as fin1,open('lexicon.txt','rt',encoding='utf-8') as fin2,\
open('test_cail.lexiconf','w',newline='',encoding='utf-8') as ftrs:
    lex=fin2.read().split()
    print(lex)
    count=0
#     train_=[]
    for line1 in fin1:
        line1 = line1.strip()
#         train_.append(line1)
        if len(line1)>256:
            count+=1
       
        if not line1:
            continue  #结束则跳出循环
        res=[]
        for l in lex:
            f=re.finditer(l,line1)
            for x in f:
                if x.span()[0]<512 and x.span()[1]<512:
                    res+=[str(x.span()[0]),str(x.span()[1])]
#                     res[x.span()[0]][x.span()[1]]='1'
#                 for i in range(x.span()[0],x.span()[1]):
#                     if i<512:
#                         res[i+1]='1'
        assert len(res)%2==0
                
        ftrs.write(' '.join(res)+'\n')
            
        
#         for temp in line2:
#             t=temp.split()
#             e1s=int(t[0])
#             e1e=int(t[1])
#             e2s=int(t[2])
#             e2e=int(t[3])
#             if t[4]=='sell_drugs_to' or t[4]=='provide_shelter_for':
#                 res[e1s]='B'
#                 res[e2s]='B'
#                 for i in range(e1s+1,e1e+1):
#                     res[i]='I'
#                 for i in range(e2s+1,e2e+1):
#                     res[i]='I'
#             elif t[4]=='posess' or t[4]=='traffic_in':
#                 res[e1s]='B'
#                 res[e2s]='B'
#                 for i in range(e1s+1,e1e+1):
#                     res[i]='I'
#                 for i in range(e2s+1,e2e+1):
#                     res[i]='I'
#         ftrs.write(' '.join(res)+'\n')
#         train_.append(line1)
#         print(line1,line2,res)
#         break


['尼可待因', '倍它羟基芬太尼', '克醋托啡', '布桂嗪', '地唑辛', '羟基四氢甲基二苯吡喃', '舒芬太尼', '地芬诺酯', '丁丙诺啡', '白色晶体', '丙吡胺', '冰毒', '氯胺酮', '卡西酮', '强痛定', '阿法甲基硫代芬太尼', '伪麻黄碱', '麻果', '白色粉末状毒品', '艾司唑仑', '埃托啡', '盐酸二氢埃托啡', '“麻果”', '安钠咖', '大麻油', '麻黄碱', '氯氮卓', '阿法甲基芬太尼', '甲基苯丙胺晶体', 'k粉', '甲卡西酮', '左啡诺', '乙酰阿法甲基芬太尼', '仲氟代芬太尼', '二甲氧基安非他明', '醋酸酐', '阿片', '二乙基色胺', '油', '辣面', '安那度尔', '芬太尼', 'K粉', '1-苯乙基-4-苯基-4-4哌啶丙盐酸', '3-甲基硫代芬太尼', '度冷丁', '阿芬太尼', '3-甲基芬太尼', '咖啡因', '福尔可定', '“白粉”', '甲基苯丙胺', '二氢埃托啡', '去甲吗啡', '二乙酰吗啡', '凯托米酮', '杜度冷丁', '麻古', '溴西泮等', '非那西汀', '大麻脂', '苯丙胺', '三氯甲烷', '瑞芬太尼', '去甲可待因', '尼美西泮', '鸦片', '大麻烟', '芬纳西泮', '罂粟壳', '乙基吗啡', '甲基苯丙胺片剂', '盐酸二氢埃托啡片', '倍它羟基-3-甲基芬太尼', '尼二可待因', '镇痛新', '布托啡诺', '海洛因', '醋氢可待因', '替利定', '可卡因', '甲基苯丙胺（冰毒）', '乙醚', '麻古丸', '大麻', '喷他佐辛', '基苯丙胺片剂', '氢吗啡醇', '吗啡', '“K粉”', '羟吗啡酮', '亚甲基二氧甲基苯丙胺', '地西泮', '吗啉乙基吗啡', '四氢大麻酚', '神仙水', '美沙酮', '氢可酮', '可待因', '硫代芬太尼', '羟考酮', '地索吗啡', '安眠酮', '哌替啶', '麻黄浸膏粉', '丙氧芬', '半粒麻古', '布莱丙胺', '大麻叶', '二亚甲基双氧安非他明', '麻黄浸膏', '白色晶体状物质', '1-甲基-4-苯基-4-哌啶丙盐酸', '氢吗啡酮', '三唑仑', '双氧可待因']
